# **There are two questions at the end of this notebook. Each one is for 5 marks. Question 3 is just for practice and not for the evaultion.**

In [1]:
import pandas as pd
import numpy as np
import torch

Loading the dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

t_indep=pd.read_csv('/content/drive/My Drive/train.csv.csv')
t_indep=t_indep.drop(t_indep.columns[[0]],axis=1)
t_dep=pd.read_csv('/content/drive/My Drive/test.csv.csv')
t_dep=t_dep.drop(t_dep.columns[[0]],axis=1)

Mounted at /content/drive


In [7]:
print(t_indep)
print(t_dep)

      Age  SibSp  Parch   LogFare  Sex_male  Sex_female  Pclass_1  Pclass_2  \
0    22.0      1      0  2.110213         1           0         0         0   
1    38.0      1      0  4.280593         0           1         1         0   
2    26.0      0      0  2.188856         0           1         0         0   
3    35.0      1      0  3.990834         0           1         1         0   
4    35.0      0      0  2.202765         1           0         0         0   
..    ...    ...    ...       ...       ...         ...       ...       ...   
886  27.0      0      0  2.639057         1           0         0         1   
887  19.0      0      0  3.433987         0           1         1         0   
888  24.0      1      2  3.196630         0           1         0         0   
889  26.0      0      0  3.433987         1           0         1         0   
890  32.0      0      0  2.169054         1           0         0         0   

     Pclass_3  Embarked_C  Embarked_Q  Embarked_S  

So PyTorch tensors allow us to use gradients and make life easier, so we'll convert these dataframe values into those


In [8]:
t_dep=torch.tensor(t_dep.values,dtype=torch.float)
t_indep=torch.tensor(t_indep.values,dtype=torch.float)

In [9]:
print(t_indep.size())

torch.Size([891, 12])


Setting up a Linear Model

In [10]:
torch.manual_seed(42)

n_coeff=t_indep.size()[1]
coeffs=torch.rand(n_coeff)-0.5 #random values in range -0.5 to 0.5
coeffs


tensor([ 0.3823,  0.4150, -0.1171,  0.4593, -0.1096,  0.1009, -0.2434,  0.2936,
         0.4408, -0.3668,  0.4346,  0.0936])

Normalizing values in each column

This is done to prevent any one column dominating the prediction results, since a linear model is row*coeffs, very high initial values in some columns will lead to some columns dominating the final answer which we don't want

In [21]:
vals,indices = t_indep.max(dim =0)
t_indep = t_indep / vals  #v cool line of code, think about why


In [25]:
preds=(t_indep*coeffs).sum(axis=1)

In [26]:
preds[0:10]

tensor([0.7371, 0.0391, 0.9206, 0.4639, 0.7542, 1.0459, 0.2906, 0.7982, 0.9089,
        0.3994])

In [27]:
loss=torch.abs(preds-t_dep).mean()
loss

tensor(0.5584)

Making functions for loss and predictions

In [28]:
def pred(coeffs,indeps):
  return (indeps*coeffs).sum(axis=1)

def calc_loss(coeffs,indeps,deps):
  return torch.abs(pred(coeffs,indeps)-deps).mean()

Doing gradient descent

In [29]:
coeffs.requires_grad_() #enables gradients for coeffs tensor

tensor([ 0.3823,  0.4150, -0.1171,  0.4593, -0.1096,  0.1009, -0.2434,  0.2936,
         0.4408, -0.3668,  0.4346,  0.0936], requires_grad=True)

In [30]:
loss=calc_loss(coeffs,t_indep,t_dep)
loss

tensor(0.5584, grad_fn=<MeanBackward0>)

In [31]:
loss.backward() #calculates gradients

In [32]:
coeffs.grad

tensor([ 0.0775,  0.0306,  0.0236,  0.1006,  0.1234,  0.1237, -0.0350,  0.0523,
         0.2297, -0.0406,  0.0847,  0.2029])

Each time you call loss.backwards, newly calculated gradients are accumulated (or added to current gradients)

We use tensor.grad.zero_() to make the gradients zero after each step

In [34]:
coeffs.grad.zero_()
loss=calc_loss(coeffs,t_indep,t_dep)
loss.backward()
with torch.no_grad():
  coeffs.sub_(coeffs.grad*0.1)
  coeffs.grad.zero_()
  print(calc_loss(coeffs,t_indep,t_dep))

tensor(0.5331)


Making functions for this

In [35]:
def update_coeffs(coeffs,lr):
  coeffs.sub_(coeffs.grad*lr)
  coeffs.grad.zero_()

In [36]:
def one_epoch(coeffs,lr):
  loss=calc_loss(coeffs,t_indep,t_dep)
  loss.backward()
  with torch.no_grad():
    update_coeffs(coeffs,lr)
    print(f"{loss:.3f}", end="; ")


In [37]:
def init_coeffs():
  return (torch.rand(n_coeff)-0.5).requires_grad_()


In [42]:
def train_model(epochs=3000, lr=0.01):
    torch.manual_seed(442)
    coeffs = init_coeffs()
    for i in range(epochs): one_epoch(coeffs, lr=lr)
    return coeffs

In [43]:
train_model()

0.531; 0.531; 0.530; 0.529; 0.528; 0.528; 0.527; 0.526; 0.526; 0.525; 0.524; 0.523; 0.523; 0.522; 0.521; 0.521; 0.520; 0.519; 0.519; 0.518; 0.517; 0.517; 0.516; 0.515; 0.515; 0.514; 0.513; 0.513; 0.512; 0.511; 0.511; 0.510; 0.509; 0.509; 0.508; 0.508; 0.507; 0.506; 0.506; 0.505; 0.505; 0.504; 0.503; 0.503; 0.502; 0.502; 0.501; 0.500; 0.500; 0.499; 0.499; 0.498; 0.497; 0.497; 0.496; 0.496; 0.495; 0.494; 0.494; 0.493; 0.493; 0.492; 0.492; 0.491; 0.490; 0.490; 0.489; 0.489; 0.488; 0.487; 0.487; 0.486; 0.486; 0.485; 0.485; 0.484; 0.483; 0.483; 0.482; 0.482; 0.481; 0.481; 0.480; 0.479; 0.479; 0.478; 0.478; 0.477; 0.476; 0.476; 0.475; 0.475; 0.474; 0.474; 0.473; 0.472; 0.472; 0.471; 0.471; 0.470; 0.470; 0.469; 0.468; 0.468; 0.467; 0.467; 0.466; 0.466; 0.465; 0.464; 0.464; 0.463; 0.463; 0.462; 0.462; 0.461; 0.460; 0.460; 0.459; 0.459; 0.458; 0.458; 0.457; 0.456; 0.456; 0.455; 0.455; 0.454; 0.454; 0.453; 0.452; 0.452; 0.451; 0.451; 0.450; 0.450; 0.449; 0.449; 0.448; 0.448; 0.447; 0.446; 0.446;

tensor([ 0.0017,  0.0017,  0.0012, -0.0011, -0.3341, -0.3371,  0.3293,  0.3279,
         0.3306,  0.0125,  0.0124,  0.0154], requires_grad=True)

In [44]:
print(coeffs.grad) #shows us that the model has pretty much converged

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


Building a full neural network

In [125]:
import torch.nn.functional as F

def preds(coeffs, indeps):
    l1,l2,const = coeffs
    res = F.relu(indeps@l1) # '@' is an optimized matrix product in python
    res = res@l2 + const
    return torch.sigmoid(res)



In [119]:
def calc_loss(coeffs,indeps,deps):
  return torch.abs(preds(coeffs,indeps)-deps).mean()

In [120]:
def one_epoch(coeffs,lr):
  loss=calc_loss(coeffs,t_indep,t_dep)
  loss.backward()
  with torch.no_grad():
    update_coeffs(coeffs,lr)
    print(f"{loss:.3f}", end="; ")

In [121]:
def train_model(epochs=300, lr=0.1):
    torch.manual_seed(442)
    coeffs = init_coeffs()
    for i in range(epochs): one_epoch(coeffs, lr=lr)
    return coeffs

In [122]:
def init_coeffs(n_hidden=60):
    layer1 = (torch.rand(n_coeff, n_hidden)-0.5)/n_hidden
    layer2 = torch.rand(n_hidden, 1)-0.3
    const = torch.rand(1)[0]
    return layer1.requires_grad_(),layer2.requires_grad_(),const.requires_grad_()

In [123]:
def update_coeffs(coeffs, lr=0.1):
    for layer in coeffs:
        layer.sub_(layer.grad * lr)
        layer.grad.zero_()

In [126]:
coeffs=train_model()
# update_coeffs(l1)


0.524; 0.520; 0.518; 0.516; 0.514; 0.512; 0.510; 0.508; 0.506; 0.504; 0.501; 0.499; 0.497; 0.494; 0.492; 0.490; 0.487; 0.485; 0.483; 0.480; 0.478; 0.476; 0.473; 0.471; 0.468; 0.466; 0.463; 0.461; 0.459; 0.456; 0.454; 0.451; 0.449; 0.446; 0.444; 0.441; 0.439; 0.436; 0.434; 0.431; 0.429; 0.426; 0.424; 0.421; 0.418; 0.416; 0.413; 0.411; 0.408; 0.405; 0.403; 0.400; 0.397; 0.395; 0.392; 0.389; 0.386; 0.384; 0.381; 0.378; 0.375; 0.373; 0.370; 0.367; 0.364; 0.362; 0.359; 0.357; 0.354; 0.351; 0.349; 0.346; 0.344; 0.341; 0.339; 0.336; 0.334; 0.332; 0.329; 0.327; 0.325; 0.323; 0.321; 0.318; 0.316; 0.314; 0.312; 0.310; 0.309; 0.307; 0.305; 0.303; 0.301; 0.300; 0.298; 0.296; 0.295; 0.293; 0.292; 0.290; 0.289; 0.287; 0.286; 0.285; 0.283; 0.282; 0.281; 0.279; 0.278; 0.277; 0.276; 0.275; 0.274; 0.273; 0.272; 0.271; 0.270; 0.269; 0.268; 0.267; 0.266; 0.265; 0.264; 0.263; 0.263; 0.262; 0.261; 0.260; 0.259; 0.259; 0.258; 0.257; 0.257; 0.256; 0.255; 0.255; 0.254; 0.253; 0.253; 0.252; 0.252; 0.251; 0.251;

In [79]:
def acc(coeffs , t_indep , t_dep): return (t_dep.bool()==(preds(coeffs,t_indep)>0.5)).float().mean()

In [80]:
acc(coeffs , t_indep , t_dep)

tensor(0.7868)

Evaluative Component:
1. Create a train/test split from the dataset

2. Evaluate the testing split and it's accuracy

optional: 3. Make the neural network 3 layer and evaluate accuracy

In [131]:
full_data = pd.read_csv("/content/drive/My Drive/train.csv.csv")
full_res = pd.read_csv("/content/drive/My Drive/test.csv.csv")
full_data = full_data.drop(full_data.columns[[0]] , axis = 1)
full_res = full_res.drop(full_res.columns[[0]] , axis = 1)

In [132]:
full_data = torch.tensor(full_data.values , dtype = float)
full_res = torch.tensor(full_res.values , dtype = float)

Creating a 80 by 20 split for train and test data respectively

In [133]:
ratio = int(0.8*len(full_data))
train_set = full_data[:ratio]
test_set = full_data[ratio:]

train_set_res = full_res[:ratio]
test_set_res = full_res[ratio:]

Accuracy on test_set


In [134]:
acc(coeffs , test_set.float() , test_set_res.float())

tensor(0.6536)